In [1]:
import pandas as pd
import numpy as np
import gc

import warnings
warnings.filterwarnings('ignore') #Supress unnecessary warnings for readability and cleaner presentation

In [2]:
datadir = '/data/Instacart/'

In [3]:
# load files
orders = pd.read_csv(datadir + 'orders.csv')
prior = pd.read_csv( datadir + 'order_products__prior.csv')
# train = pd.read_csv(datadir + 'order_products__train.csv')
orders['days'] = orders.groupby(['user_id']).days_since_prior_order.cumsum()
orders['days'].fillna(0, inplace=True)

In [4]:
orders_max = orders.groupby('user_id').days.agg({'max_order':'count', 'max_days':'max'}).reset_index()
orders = pd.merge(orders, orders_max, on ="user_id", how='left')
del orders_max
orders['order_weight'] = orders['order_number']/orders['max_order']
orders['time_weight'] = orders['days']/orders['max_days']
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days,max_order,max_days,order_weight,time_weight
0,2539329,1,prior,1,2,8,NaN,0.0,11,190.0,0.090909,0.000000
1,2398795,1,prior,2,3,7,15.0,15.0,11,190.0,0.181818,0.078947
2,473747,1,prior,3,3,12,21.0,36.0,11,190.0,0.272727,0.189474
3,2254736,1,prior,4,4,7,29.0,65.0,11,190.0,0.363636,0.342105
4,431534,1,prior,5,4,15,28.0,93.0,11,190.0,0.454545,0.489474


In [5]:
order_user_product = pd.merge(prior, orders[['order_id', 'user_id', 'order_number', 'days','order_weight', 'time_weight']],
                              on='order_id', how = 'inner')
order_user_product.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days,order_weight,time_weight
0,2,33120,1,1,202279,3,28.0,0.333333,0.154696
1,2,28985,2,1,202279,3,28.0,0.333333,0.154696
2,2,9327,3,0,202279,3,28.0,0.333333,0.154696
3,2,45918,4,1,202279,3,28.0,0.333333,0.154696
4,2,30035,5,0,202279,3,28.0,0.333333,0.154696


In [6]:
order_user_product['weights'] =  1/order_user_product['add_to_cart_order']
order_user_product.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days,order_weight,time_weight,weights
0,2,33120,1,1,202279,3,28.0,0.333333,0.154696,1.000000
1,2,28985,2,1,202279,3,28.0,0.333333,0.154696,0.500000
2,2,9327,3,0,202279,3,28.0,0.333333,0.154696,0.333333
3,2,45918,4,1,202279,3,28.0,0.333333,0.154696,0.250000
4,2,30035,5,0,202279,3,28.0,0.333333,0.154696,0.200000


In [7]:
del prior
del orders

In [8]:
gc.collect()
total_user = order_user_product.user_id.nunique()
products = order_user_product.groupby('product_id').agg({'user_id':'nunique', 'order_id':'nunique',
                                                          'weights':'mean', 'reordered':'sum'}).reset_index()
products_reordered = order_user_product.ix[order_user_product['reordered']==1].groupby('product_id').\
                     agg({'user_id':'nunique'}).reset_index()
products = pd.merge(products, products_reordered, on='product_id', how='left')
del products_reordered
products.rename(columns={'user_id_x':'user_count', 'reordered':'reordered_order_count',
                         'user_id_y' :'reordered_user_count','order_id':'order_count'},inplace=True)
products['product_user_reorder_ratio'] = products['reordered_user_count']/products['user_count']
products['product_reorder_ratio'] = products['reordered_order_count']/products['order_count']
products['product_user_ratio'] = products['user_count']/total_user

products.to_csv(datadir+'products_summary.csv', index = False)
products.head()


,product_id,user_count,weights,order_count,reordered_order_count,reordered_user_count,product_user_reorder_ratio,product_reorder_ratio,product_user_ratio
0,1,716,0.350873,1852,1136,276.0,0.385475,0.613391,0.003472
1,2,78,0.250289,90,12,8.0,0.102564,0.133333,0.000378
2,3,74,0.350600,277,203,36.0,0.486486,0.732852,0.000359
3,4,182,0.200119,329,147,64.0,0.351648,0.446809,0.000883
4,5,6,0.193929,15,9,4.0,0.666667,0.600000,0.000029


In [9]:
del products

In [10]:
gc.collect()
users = order_user_product.groupby('user_id').agg({'product_id':'nunique', 'order_id':'nunique','reordered':'sum',
                                                     'days':'count'}).reset_index()
users_reordered = order_user_product.ix[order_user_product['reordered']==1].groupby('user_id').\
                     agg({'order_id':'nunique', 'product_id':'nunique'}).reset_index()
users = pd.merge(users, users_reordered, on ='user_id', how = 'left')
del users_reordered
users.rename(columns={'product_id_x':'product_count', 'product_id_y':'reordered_product_count', 
                      'order_id_x':'order_count', 'order_id_y': 'reordered_order_count',
                      'days':'total_count', 'reordered':'total_reordered'}, inplace =True)
users['user_product_reorder_ratio'] = users['reordered_product_count']/users['product_count']
users['user_order_reorder_ratio'] = users['reordered_order_count']/np.maximum(1,users['order_count']-1)
users['user_total_reorder_ratio'] = users['total_reordered']/users['total_count']
users['average_product_per_order'] = users['total_count']/users['order_count']

users.to_csv(datadir+'users_summary.csv', index = False)
users.head()

,user_id,total_reordered,order_count,product_count,total_count,reordered_product_count,reordered_order_count,user_product_reorder_ratio,user_order_reorder_ratio,user_total_reorder_ratio,average_product_per_order
0,1,41,10,18,59,10.0,9.0,0.555556,1.000000,0.694915,5.900000
1,2,93,14,102,195,37.0,12.0,0.362745,0.923077,0.476923,13.928571
2,3,55,12,33,88,19.0,11.0,0.575758,1.000000,0.625000,7.333333
3,4,1,5,17,18,1.0,1.0,0.058824,0.250000,0.055556,3.600000
4,5,14,4,23,37,8.0,3.0,0.347826,1.000000,0.378378,9.250000


In [11]:
del users

In [12]:
gc.collect()
user_product = order_user_product.groupby(['user_id', 'product_id']).agg({'weights':['sum','mean'],'order_weight':['max','count','sum'], 
                                                           'days': ['sum', 'max','min', 'count'], 'time_weight':['max','sum']})

user_product.columns = ['_'.join(col).strip() for col in user_product.columns.values]
user_product.reset_index(inplace = True)
user_product['mean_gap'] = (user_product['days_max'] - user_product['days_min'])/np.maximum(1,user_product['days_count']-1)
user_product.to_csv(datadir +'user_product.csv', index = False)
user_product.head()

,user_id,product_id,weights_sum,weights_mean,order_weight_max,order_weight_count,order_weight_sum,time_weight_max,time_weight_sum,days_sum,days_max,days_min,days_count,mean_gap
0,1,196,8.750000,0.875000,0.909091,10,5.000000,0.926316,4.847368,921.0,176.0,0.0,10,19.555556
1,1,10258,3.033333,0.337037,0.909091,9,4.909091,0.926316,4.847368,921.0,176.0,15.0,9,20.125000
2,1,10326,0.200000,0.200000,0.454545,1,0.454545,0.489474,0.489474,93.0,93.0,93.0,1,0.000000
3,1,12427,4.277778,0.427778,0.909091,10,5.000000,0.926316,4.847368,921.0,176.0,0.0,10,19.555556
4,1,13032,0.491667,0.163889,0.909091,3,1.727273,0.926316,1.700000,323.0,176.0,15.0,3,80.500000
